In [68]:
import pandas as pd
import scipy.io as sio
from scipy.io import savemat, loadmat
from EEG_pareidolia_utils import get_pareidolia_bids, reformat_events
from biotuner_utils import *
from biotuner_offline import *
from biotuner import *
import pyACA
import mne
from mne.datasets.brainstorm import bst_raw
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import scipy.io as sio
import seaborn as sbn
import pytuning as pt

In [2]:
path = '../../pareidolia_run4.fif'
#path = 'C:/Users/Antoine/github/Data_EEG/pareidolia_run1.fif'
epochs = mne.read_epochs(path)
epochs = epochs.copy().apply_baseline((-1.5, -0.1))
#epochs2 = epochs2.crop(0.5, 7.5)
epochs = epochs.filter(0.1, 100)

Reading ../../pareidolia_run4.fif ...


<ipython-input-2-49aa83b1c626>:3: RuntimeWarning: This filename (../../pareidolia_run4.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(path)


    Found the data of interest:
        t =   -1500.00 ...    8000.00 ms
        0 CTF compensation matrices available
84 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Applying baseline correction (mode: mean)
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 33001 samples (33.001 sec)



<ipython-input-2-49aa83b1c626>:6: RuntimeWarning: filter_length (33001) is longer than the signal (9501), distortion is likely. Reduce filter length or filter a longer signal.
  epochs = epochs.filter(0.1, 100)


In [10]:
epochs.get_data()[:, 0, :].shape

(84, 9501)

In [45]:
#####COMPUTE PEAKS#####

from biotuner_offline import *
par = ['Image_on_par_high', 'Image_on_par_mid', 'Image_on_par_low']
nopar = ['Image_on_nopar_high', 'Image_on_nopar_mid', 'Image_on_nopar_low']
condition = par
ch = ['E22', 'E32', 'E33', 'E81']
dim_reduc = 'avg' #Mode could either be 'avg' or 'mode'
precision = 0.25   #Default precision is 0.125Hz
FREQ_BANDS = [[1, 3], [3, 7], [7, 12], [12, 18],[18, 30], [30, 50]]
#peaks= compute_peaks(epochs.copy(), condition, ch, FREQ_BANDS, tmin = 0.5, tmax = 7.5, precision = precision, sf =1000)
peaks= compute_peaks_avg(epochs.copy(), condition, ch, FREQ_BANDS, dim_reduc = dim_reduc, tmin = 0.5, tmax = 7.5, precision = precision, sf =1000)
peaks

Applying baseline correction (mode: mean)
(84, 114, 7001)
7000
4000
Effective window size : 4.000 (s)
0
Effective window size : 4.000 (s)
0
Effective window size : 4.000 (s)
1
Effective window size : 4.000 (s)
19
Effective window size : 4.000 (s)
37
Effective window size : 4.000 (s)
9


array([ 1. ,  3. ,  7.2, 16.8, 27.2, 32.2])

In [32]:
x, ratios = compute_peak_ratios(peaks)

In [60]:
peaks[1:]

array([ 3. ,  7.2, 16.8, 27.2, 32.2])

In [61]:
EEG_harmonics_mult(peaks[1:], 20, n_oct_up = 0)

array([[  3. ,   6. ,   9. ,  12. ,  15. ,  18. ,  21. ,  24. ,  27. ,
         30. ,  33. ,  36. ,  39. ,  42. ,  45. ,  48. ,  51. ,  54. ,
         57. ,  60. ,  63. ],
       [  7.2,  14.4,  21.6,  28.8,  36. ,  43.2,  50.4,  57.6,  64.8,
         72. ,  79.2,  86.4,  93.6, 100.8, 108. , 115.2, 122.4, 129.6,
        136.8, 144. , 151.2],
       [ 16.8,  33.6,  50.4,  67.2,  84. , 100.8, 117.6, 134.4, 151.2,
        168. , 184.8, 201.6, 218.4, 235.2, 252. , 268.8, 285.6, 302.4,
        319.2, 336. , 352.8],
       [ 27.2,  54.4,  81.6, 108.8, 136. , 163.2, 190.4, 217.6, 244.8,
        272. , 299.2, 326.4, 353.6, 380.8, 408. , 435.2, 462.4, 489.6,
        516.8, 544. , 571.2],
       [ 32.2,  64.4,  96.6, 128.8, 161. , 193.2, 225.4, 257.6, 289.8,
        322. , 354.2, 386.4, 418.6, 450.8, 483. , 515.2, 547.4, 579.6,
        611.8, 644. , 676.2]])

In [65]:
#ratios of harmonic fits

z, charm_fit = compute_peak_ratios(harmonic_fit(peaks[1:], 30, 0.1, 'mult'))
charm_fit

[1.0499999999999998,
 1.15,
 1.1666666666666667,
 1.2,
 1.277777777777778,
 1.3333333333333335,
 1.3416666666666666,
 1.4,
 1.4285714285714286,
 1.5,
 1.6666666666666667,
 1.7999999999999998,
 1.9166666666666665,
 2.0]

In [97]:
from pytuning import *
intervals     = peaks
multiplicities = [2,2,2, 2, 2, 2]
scale1        = create_euler_fokker_scale(intervals, multiplicities)
scale1

[1,
 2187/2048,
 35721/32768,
 567/512,
 9/8,
 147/128,
 321489/262144,
 5103/4096,
 81/64,
 1323/1024,
 21/16,
 45927/32768,
 729/512,
 11907/8192,
 189/128,
 3/2,
 49/32,
 6561/4096,
 107163/65536,
 1701/1024,
 27/16,
 441/256,
 7/4,
 15309/8192,
 243/128,
 3969/2048,
 63/32,
 2]

In [93]:
peaks

array([ 1. ,  3. ,  7.2, 16.8, 27.2, 32.2])

In [ ]:
x = pt.find_best_modes(scale1, 12)
x

In [69]:
create_SCL(charm_fit, 'eeg_scale_test')